# 📊 Network Usage Analysis - Interactive Dashboard
## Análisis Interactivo de Uso de Red por Hora y Edificio

**Objetivo:** Visualizar el uso de la red por hora del día con filtros interactivos para:
- Día de la semana
- Código de edificio

---

## 📦 1. Importar Librerías

In [1]:
# Librerías estándar
import pandas as pd
import numpy as np
import re
import warnings
from pathlib import Path

# Plotly para visualizaciones interactivas
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración
warnings.filterwarnings('ignore')

# Cargar utilidades
import sys
utils_path = Path('../starter_kits/utils').resolve()
if str(utils_path) not in sys.path:
    sys.path.append(str(utils_path))

from data_loader import (
    load_clients,
    AP_NAME_PATTERN
)

print("✅ Librerías cargadas correctamente")

✅ Librerías cargadas correctamente


## 📂 2. Cargar Datos

Cargamos el dataset completo de clientes desde `realData/clients`

In [2]:
# Cargar datos de clientes
df_clients = load_clients(
    data_dir="../realData/clients",
    max_files=None,  # Cambiar a un número menor para pruebas rápidas
    verbose=True
)

print(f"\n🎯 Total de registros cargados: {len(df_clients):,}")
print(f"📅 Rango de fechas: {df_clients['timestamp'].min()} - {df_clients['timestamp'].max()}")

📁 Encontrados 299 archivos en ../realData/clients
📊 Cargando todos archivos...
   Procesados 10/299 archivos... (15581 registros)
   Procesados 20/299 archivos... (129244 registros)
   Procesados 20/299 archivos... (129244 registros)
   Procesados 30/299 archivos... (260458 registros)
   Procesados 30/299 archivos... (260458 registros)
   Procesados 40/299 archivos... (328976 registros)
   Procesados 50/299 archivos... (338274 registros)
   Procesados 40/299 archivos... (328976 registros)
   Procesados 50/299 archivos... (338274 registros)
   Procesados 60/299 archivos... (422585 registros)
   Procesados 60/299 archivos... (422585 registros)
   Procesados 70/299 archivos... (538256 registros)
   Procesados 70/299 archivos... (538256 registros)
   Procesados 80/299 archivos... (601490 registros)
   Procesados 90/299 archivos... (610072 registros)
   Procesados 80/299 archivos... (601490 registros)
   Procesados 90/299 archivos... (610072 registros)
   Procesados 100/299 archivos... (622

## 🏢 3. Extraer Código de Edificio

Extraemos el código del edificio del nombre del AP (ej: AP-CEDU33 → CEDU)

In [3]:
# Patrón para extraer código de edificio
pattern = re.compile(AP_NAME_PATTERN)

def extract_building_code(ap_name):
    """Extrae el código del edificio del nombre del AP"""
    if not isinstance(ap_name, str):
        return 'UNKNOWN'
    match = pattern.match(ap_name)
    return match.group(1) if match else 'UNKNOWN'

# Aplicar extracción
df_clients['building_code'] = df_clients['associated_device_name'].apply(extract_building_code)

# Mostrar estadísticas
print("🏢 Edificios únicos encontrados:")
building_counts = df_clients['building_code'].value_counts()
print(f"\nTotal de edificios: {len(building_counts)}")
print(f"\nTop 10 edificios por uso:")
print(building_counts.head(10))

🏢 Edificios únicos encontrados:

Total de edificios: 50

Top 10 edificios por uso:
building_code
LLET     268801
CIEN     226885
ETSE     135375
CEDU     112312
VET       97854
DRET      91406
AULAJ     72255
MED       69450
ECON      62425
CCOM      59816
Name: count, dtype: int64


## 📅 4. Preparar Datos Temporales

Agregamos información adicional de tiempo para análisis

In [24]:
# Verificar que tenemos las columnas necesarias
if 'day_of_week' not in df_clients.columns:
    df_clients['day_of_week'] = df_clients['timestamp'].dt.day_name()

if 'hour' not in df_clients.columns:
    df_clients['hour'] = df_clients['timestamp'].dt.hour

# Crear categorías de día (laborable vs fin de semana)
df_clients['day_type'] = df_clients['day_of_week'].apply(
    lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday'
)

print("✅ Datos temporales preparados")
print(f"\nDías de la semana disponibles:")
print(df_clients['day_of_week'].value_counts().sort_index())

✅ Datos temporales preparados

Días de la semana disponibles:
day_of_week
Friday       266966
Monday       301411
Saturday      16501
Sunday        12333
Thursday     603726
Tuesday      316273
Wednesday    277047
Name: count, dtype: int64


## 📊 5. Análisis de Uso de Red por Hora

### 5.1 Vista General - Todas las Conexiones

In [25]:
# Calcular dispositivos conectados por hora (promedio)
# Primero agrupamos por fecha y hora para contar conexiones por día
df_clients['date'] = df_clients['timestamp'].dt.date
hourly_usage_by_date = df_clients.groupby(['date', 'hour']).size().reset_index(name='device_count')

# Luego promediamos por hora a través de todos los días
hourly_usage = hourly_usage_by_date.groupby('hour')['device_count'].mean().reset_index()
hourly_usage.columns = ['hour', 'device_count']

# Crear gráfico interactivo con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=hourly_usage['hour'],
    y=hourly_usage['device_count'],
    mode='lines+markers',
    name='Dispositivos Conectados (Promedio)',
    line=dict(color='coral', width=3),
    marker=dict(size=8),
    fill='tozeroy',
    fillcolor='rgba(255, 127, 80, 0.3)',
    hovertemplate='<b>Hora:</b> %{x}:00<br><b>Conexiones (promedio):</b> %{y:.1f}<extra></extra>'
))

fig.update_layout(
    title='⏰ Dispositivos Conectados por Hora del Día - Vista General (Promedio)',
    xaxis_title='Hora del Día (0-23)',
    yaxis_title='Número Promedio de Dispositivos',
    hovermode='x unified',
    template='plotly_white',
    height=500,
    font=dict(size=12)
)

fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)

fig.show()

# Mostrar hora pico
peak_hour = hourly_usage.loc[hourly_usage['device_count'].idxmax()]
print(f"\n🕐 Hora pico: {int(peak_hour['hour'])}:00 con {peak_hour['device_count']:.1f} dispositivos conectados (promedio)")


🕐 Hora pico: 7:00 con 33077.0 dispositivos conectados (promedio)


### 5.2 Por Día de la Semana

In [26]:
# Calcular uso por hora y día de la semana (promedio)
# Primero agrupamos por fecha, hora y día de la semana
hourly_by_day_date = df_clients.groupby(['date', 'hour', 'day_of_week']).size().reset_index(name='device_count')

# Luego promediamos por hora y día de la semana
hourly_by_day = hourly_by_day_date.groupby(['hour', 'day_of_week'])['device_count'].mean().reset_index()

# Orden correcto de días
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hourly_by_day['day_of_week'] = pd.Categorical(hourly_by_day['day_of_week'], categories=day_order, ordered=True)
hourly_by_day = hourly_by_day.sort_values(['day_of_week', 'hour'])

# Crear gráfico interactivo
fig = px.line(
    hourly_by_day,
    x='hour',
    y='device_count',
    color='day_of_week',
    title='📅 Uso de Red por Hora - Comparación por Día de la Semana (Promedio)',
    labels={
        'hour': 'Hora del Día',
        'device_count': 'Número Promedio de Dispositivos',
        'day_of_week': 'Día de la Semana'
    },
    template='plotly_white',
    height=600
)

fig.update_traces(mode='lines+markers', line=dict(width=2), marker=dict(size=6))
fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)
fig.update_layout(hovermode='x unified', font=dict(size=12))

fig.show()

### 5.3 Por Edificio (Top 10)

In [27]:
# Identificar top 10 edificios
top_buildings = df_clients['building_code'].value_counts().head(10).index.tolist()

# Filtrar datos para top edificios
df_top_buildings = df_clients[df_clients['building_code'].isin(top_buildings)]

# Calcular uso por hora y edificio (promedio)
# Primero agrupamos por fecha, hora y edificio
hourly_by_building_date = df_top_buildings.groupby(['date', 'hour', 'building_code']).size().reset_index(name='device_count')

# Luego promediamos por hora y edificio
hourly_by_building = hourly_by_building_date.groupby(['hour', 'building_code'])['device_count'].mean().reset_index()

# Crear gráfico interactivo
fig = px.line(
    hourly_by_building,
    x='hour',
    y='device_count',
    color='building_code',
    title='🏢 Uso de Red por Hora - Top 10 Edificios (Promedio)',
    labels={
        'hour': 'Hora del Día',
        'device_count': 'Número Promedio de Dispositivos',
        'building_code': 'Código de Edificio'
    },
    template='plotly_white',
    height=600
)

fig.update_traces(mode='lines+markers', line=dict(width=2), marker=dict(size=6))
fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)
fig.update_layout(hovermode='x unified', font=dict(size=12))

fig.show()

## 🎛️ 6. Dashboard Interactivo con Filtros

### 6.1 Filtro por Día de la Semana

In [28]:
# Preparar datos con todos los días (promedio)
# Primero agrupamos por fecha, hora y día de la semana
hourly_by_day_all_date = df_clients.groupby(['date', 'hour', 'day_of_week']).size().reset_index(name='device_count')

# Luego promediamos por hora y día de la semana
hourly_by_day_all = hourly_by_day_all_date.groupby(['hour', 'day_of_week'])['device_count'].mean().reset_index()

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hourly_by_day_all['day_of_week'] = pd.Categorical(
    hourly_by_day_all['day_of_week'], 
    categories=day_order, 
    ordered=True
)
hourly_by_day_all = hourly_by_day_all.sort_values(['day_of_week', 'hour'])

# Crear figura con botones para filtrar por día
fig = go.Figure()

# Agregar traza para cada día
for day in day_order:
    day_data = hourly_by_day_all[hourly_by_day_all['day_of_week'] == day]
    fig.add_trace(go.Scatter(
        x=day_data['hour'],
        y=day_data['device_count'],
        mode='lines+markers',
        name=day,
        visible=True if day == 'Monday' else False,
        line=dict(width=3),
        marker=dict(size=8),
        fill='tozeroy',
        hovertemplate=f'<b>{day}</b><br>Hora: %{{x}}:00<br>Dispositivos (promedio): %{{y:.1f}}<extra></extra>'
    ))

# Crear botones para cada día
buttons = []
for i, day in enumerate(day_order):
    visible = [False] * len(day_order)
    visible[i] = True
    buttons.append(
        dict(
            label=day,
            method='update',
            args=[{'visible': visible}]
        )
    )

# Agregar botón "Todos"
buttons.insert(0, dict(
    label='Todos',
    method='update',
    args=[{'visible': [True] * len(day_order)}]
))

fig.update_layout(
    title='📅 Uso de Red por Hora - Filtro Interactivo por Día de la Semana (Promedio)',
    xaxis_title='Hora del Día (0-23)',
    yaxis_title='Número Promedio de Dispositivos',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.17,
            xanchor='left',
            y=1.15,
            yanchor='top'
        )
    ],
    template='plotly_white',
    height=600,
    hovermode='x unified',
    font=dict(size=12)
)

fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)

fig.show()

### 6.2 Filtro por Edificio

In [29]:
# Obtener top 15 edificios para el filtro
top_15_buildings = df_clients['building_code'].value_counts().head(15).index.tolist()
df_filtered = df_clients[df_clients['building_code'].isin(top_15_buildings)]

# Preparar datos (promedio)
# Primero agrupamos por fecha, hora y edificio
hourly_by_building_all_date = df_filtered.groupby(['date', 'hour', 'building_code']).size().reset_index(name='device_count')

# Luego promediamos por hora y edificio
hourly_by_building_all = hourly_by_building_all_date.groupby(['hour', 'building_code'])['device_count'].mean().reset_index()
hourly_by_building_all = hourly_by_building_all.sort_values(['building_code', 'hour'])

# Crear figura
fig = go.Figure()

# Agregar traza para cada edificio
for i, building in enumerate(top_15_buildings):
    building_data = hourly_by_building_all[hourly_by_building_all['building_code'] == building]
    fig.add_trace(go.Scatter(
        x=building_data['hour'],
        y=building_data['device_count'],
        mode='lines+markers',
        name=building,
        visible=True if i == 0 else False,
        line=dict(width=3),
        marker=dict(size=8),
        fill='tozeroy',
        hovertemplate=f'<b>{building}</b><br>Hora: %{{x}}:00<br>Dispositivos (promedio): %{{y:.1f}}<extra></extra>'
    ))

# Crear botones para cada edificio
buttons = []
for i, building in enumerate(top_15_buildings):
    visible = [False] * len(top_15_buildings)
    visible[i] = True
    buttons.append(
        dict(
            label=building,
            method='update',
            args=[{'visible': visible}]
        )
    )

# Agregar botón "Todos"
buttons.insert(0, dict(
    label='Todos',
    method='update',
    args=[{'visible': [True] * len(top_15_buildings)}]
))

fig.update_layout(
    title='🏢 Uso de Red por Hora - Filtro Interactivo por Edificio (Top 15, Promedio)',
    xaxis_title='Hora del Día (0-23)',
    yaxis_title='Número Promedio de Dispositivos',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.17,
            xanchor='left',
            y=1.15,
            yanchor='top'
        )
    ],
    template='plotly_white',
    height=600,
    hovermode='x unified',
    font=dict(size=12)
)

fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)

fig.show()

### 6.3 Dashboard Completo - Filtros Combinados (Día + Edificio)

In [30]:
# Preparar datos para combinación día + edificio (promedio)
df_filtered = df_clients[df_clients['building_code'].isin(top_15_buildings)]

# Primero agrupamos por fecha, hora, día de la semana y edificio
combined_data_date = df_filtered.groupby(['date', 'hour', 'day_of_week', 'building_code']).size().reset_index(name='device_count')

# Luego promediamos por hora, día de la semana y edificio
combined_data = combined_data_date.groupby(['hour', 'day_of_week', 'building_code'])['device_count'].mean().reset_index()

# Ordenar días
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
combined_data['day_of_week'] = pd.Categorical(
    combined_data['day_of_week'], 
    categories=day_order, 
    ordered=True
)
combined_data = combined_data.sort_values(['day_of_week', 'building_code', 'hour'])

# Crear figura con subplots
fig = go.Figure()

# Agregar trazas para cada combinación día + edificio
trace_idx = 0
trace_map = {}  # Mapear (día, edificio) -> índice de traza

for day in day_order:
    for building in top_15_buildings:
        data_subset = combined_data[
            (combined_data['day_of_week'] == day) & 
            (combined_data['building_code'] == building)
        ]
        
        if len(data_subset) > 0:
            fig.add_trace(go.Scatter(
                x=data_subset['hour'],
                y=data_subset['device_count'],
                mode='lines+markers',
                name=f'{day} - {building}',
                visible=True if (day == 'Monday' and building == top_15_buildings[0]) else False,
                line=dict(width=3),
                marker=dict(size=8),
                fill='tozeroy',
                hovertemplate=f'<b>{day} - {building}</b><br>Hora: %{{x}}:00<br>Dispositivos (promedio): %{{y:.1f}}<extra></extra>'
            ))
            trace_map[(day, building)] = trace_idx
            trace_idx += 1

# Crear botones para días
day_buttons = []
for day in day_order:
    visible = [False] * trace_idx
    for building in top_15_buildings:
        if (day, building) in trace_map:
            visible[trace_map[(day, building)]] = True
    
    day_buttons.append(
        dict(
            label=day,
            method='update',
            args=[{'visible': visible}]
        )
    )

# Agregar botón "Todos los días"
day_buttons.insert(0, dict(
    label='Todos los días',
    method='update',
    args=[{'visible': [True] * trace_idx}]
))

# Crear botones para edificios
building_buttons = []
for building in top_15_buildings:
    visible = [False] * trace_idx
    for day in day_order:
        if (day, building) in trace_map:
            visible[trace_map[(day, building)]] = True
    
    building_buttons.append(
        dict(
            label=building,
            method='update',
            args=[{'visible': visible}]
        )
    )

# Agregar botón "Todos los edificios"
building_buttons.insert(0, dict(
    label='Todos los edificios',
    method='update',
    args=[{'visible': [True] * trace_idx}]
))

fig.update_layout(
    title='🎛️ Dashboard Completo - Uso de Red con Filtros Combinados (Promedio)',
    xaxis_title='Hora del Día (0-23)',
    yaxis_title='Número Promedio de Dispositivos',
    updatemenus=[
        # Dropdown para días
        dict(
            buttons=day_buttons,
            direction='down',
            showactive=True,
            x=0.17,
            xanchor='left',
            y=1.20,
            yanchor='top',
            bgcolor='lightblue',
            bordercolor='blue',
            font=dict(size=11)
        ),
        # Dropdown para edificios
        dict(
            buttons=building_buttons,
            direction='down',
            showactive=True,
            x=0.40,
            xanchor='left',
            y=1.20,
            yanchor='top',
            bgcolor='lightgreen',
            bordercolor='green',
            font=dict(size=11)
        )
    ],
    annotations=[
        dict(text='Día de la semana:', x=0, xref='paper', xanchor='right',
             y=1.20, yref='paper', yanchor='top', showarrow=False, font=dict(size=12)),
        dict(text='Edificio:', x=0.27, xref='paper', xanchor='right',
             y=1.20, yref='paper', yanchor='top', showarrow=False, font=dict(size=12))
    ],
    template='plotly_white',
    height=650,
    hovermode='x unified',
    font=dict(size=12)
)

fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)

fig.show()

print("\n✅ Dashboard interactivo creado exitosamente!")
print("\n📝 Instrucciones:")
print("   - Usa el primer dropdown para filtrar por día de la semana")
print("   - Usa el segundo dropdown para filtrar por edificio")
print("   - Selecciona 'Todos' en cualquier filtro para ver todas las opciones")
print("   - Puedes hacer zoom, pan y hover sobre el gráfico para más detalles")
print("\n⚠️  Nota: Los valores mostrados son PROMEDIOS de múltiples semanas/días")


✅ Dashboard interactivo creado exitosamente!

📝 Instrucciones:
   - Usa el primer dropdown para filtrar por día de la semana
   - Usa el segundo dropdown para filtrar por edificio
   - Selecciona 'Todos' en cualquier filtro para ver todas las opciones
   - Puedes hacer zoom, pan y hover sobre el gráfico para más detalles

⚠️  Nota: Los valores mostrados son PROMEDIOS de múltiples semanas/días


## 📈 7. Análisis Adicional - Heatmap por Día y Hora

In [31]:
# Crear heatmap de uso por día y hora (promedio)
# Primero agrupamos por fecha, día de la semana y hora
heatmap_data_date = df_clients.groupby(['date', 'day_of_week', 'hour']).size().reset_index(name='device_count')

# Luego promediamos por día de la semana y hora
heatmap_data = heatmap_data_date.groupby(['day_of_week', 'hour'])['device_count'].mean().reset_index()

# Pivotar para heatmap
heatmap_pivot = heatmap_data.pivot(index='day_of_week', columns='hour', values='device_count')
heatmap_pivot = heatmap_pivot.reindex(day_order)

# Crear heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    colorscale='YlOrRd',
    hovertemplate='<b>%{y}</b><br>Hora: %{x}:00<br>Dispositivos (promedio): %{z:.1f}<extra></extra>'
))

fig.update_layout(
    title='🔥 Heatmap de Uso de Red - Día vs Hora (Promedio)',
    xaxis_title='Hora del Día',
    yaxis_title='Día de la Semana',
    template='plotly_white',
    height=500,
    font=dict(size=12)
)

fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)

fig.show()

## 🏢 8. Heatmap por Edificio y Hora (Top 10)

In [32]:
# Filtrar top 10 edificios
top_10_buildings = df_clients['building_code'].value_counts().head(10).index.tolist()
df_top_10 = df_clients[df_clients['building_code'].isin(top_10_buildings)]

# Crear heatmap (promedio)
# Primero agrupamos por fecha, edificio y hora
heatmap_building_date = df_top_10.groupby(['date', 'building_code', 'hour']).size().reset_index(name='device_count')

# Luego promediamos por edificio y hora
heatmap_building = heatmap_building_date.groupby(['building_code', 'hour'])['device_count'].mean().reset_index()

heatmap_building_pivot = heatmap_building.pivot(index='building_code', columns='hour', values='device_count')

# Ordenar por uso total
heatmap_building_pivot['total'] = heatmap_building_pivot.sum(axis=1)
heatmap_building_pivot = heatmap_building_pivot.sort_values('total', ascending=False)
heatmap_building_pivot = heatmap_building_pivot.drop('total', axis=1)

# Crear figura
fig = go.Figure(data=go.Heatmap(
    z=heatmap_building_pivot.values,
    x=heatmap_building_pivot.columns,
    y=heatmap_building_pivot.index,
    colorscale='Viridis',
    hovertemplate='<b>%{y}</b><br>Hora: %{x}:00<br>Dispositivos (promedio): %{z:.1f}<extra></extra>'
))

fig.update_layout(
    title='🏢 Heatmap de Uso por Edificio y Hora - Top 10 Edificios (Promedio)',
    xaxis_title='Hora del Día',
    yaxis_title='Código de Edificio',
    template='plotly_white',
    height=600,
    font=dict(size=12)
)

fig.update_xaxes(tickmode='linear', tick0=0, dtick=1)

fig.show()

## 📊 9. Resumen de Insights

Genera automáticamente insights basados en los datos

In [33]:
# Calcular estadísticas clave
total_devices = df_clients['macaddr'].nunique()
total_connections = len(df_clients)
total_buildings = df_clients['building_code'].nunique()

# Hora pico general
hourly_total = df_clients.groupby('hour').size()
peak_hour = hourly_total.idxmax()
peak_devices = hourly_total.max()

# Día más activo
daily_total = df_clients.groupby('day_of_week').size()
peak_day = daily_total.idxmax()
peak_day_devices = daily_total.max()

# Edificio más usado
building_total = df_clients.groupby('building_code').size()
top_building = building_total.idxmax()
top_building_devices = building_total.max()

# Diferencia entre día laborable y fin de semana
weekday_avg = df_clients[df_clients['day_type'] == 'Weekday'].groupby('hour').size().mean()
weekend_avg = df_clients[df_clients['day_type'] == 'Weekend'].groupby('hour').size().mean()
weekday_vs_weekend = ((weekday_avg - weekend_avg) / weekend_avg) * 100

# Imprimir resumen
print("="*70)
print("📊 RESUMEN DE INSIGHTS - ANÁLISIS DE USO DE RED")
print("="*70)
print(f"\n📱 Dispositivos únicos: {total_devices:,}")
print(f"📡 Total de conexiones: {total_connections:,}")
print(f"🏢 Edificios únicos: {total_buildings}")
print(f"\n🕐 Hora pico: {peak_hour}:00 con {peak_devices:,} dispositivos")
print(f"📅 Día más activo: {peak_day} con {peak_day_devices:,} conexiones")
print(f"🏢 Edificio más usado: {top_building} con {top_building_devices:,} conexiones")
print(f"\n📈 Diferencia Día Laborable vs Fin de Semana: {weekday_vs_weekend:+.1f}%")
print(f"   (Los días laborables tienen {abs(weekday_vs_weekend):.1f}% {'más' if weekday_vs_weekend > 0 else 'menos'} actividad)")
print("\n" + "="*70)

📊 RESUMEN DE INSIGHTS - ANÁLISIS DE USO DE RED

📱 Dispositivos únicos: 69,369
📡 Total de conexiones: 1,865,407
🏢 Edificios únicos: 50

🕐 Hora pico: 9.0:00 con 308,252 dispositivos
📅 Día más activo: Thursday con 603,726 conexiones
🏢 Edificio más usado: LLET con 268,801 conexiones

📈 Diferencia Día Laborable vs Fin de Semana: +6022.7%
   (Los días laborables tienen 6022.7% más actividad)



## 🎯 10. Conclusiones

Este notebook proporciona un análisis interactivo completo del uso de la red WiFi en el campus de la UAB:

- ✅ **Carga completa del dataset** desde realData/clients
- ✅ **Extracción de códigos de edificio** desde nombres de APs
- ✅ **Visualizaciones interactivas con Plotly** que permiten explorar:
  - Patrones temporales por hora
  - Comparaciones entre días de la semana
  - Análisis por edificio
- ✅ **Filtros interactivos** para personalizar la vista según necesidades
- ✅ **Heatmaps** para identificar patrones de uso rápidamente
- ✅ **Insights automáticos** con estadísticas clave

### 🚀 Próximos Pasos

Puedes extender este análisis:
1. Comparar diferentes períodos de tiempo (meses, semestres)
2. Analizar velocidad de conexión por edificio/hora
3. Identificar problemas de conectividad por zona y horario
4. Crear predicciones de uso futuro
5. Exportar los gráficos como HTML para compartir

---

**🏴‍☠️ UAB THE HACK! 2025 - ¡Análisis completado! 🚀**